# Ch.04 機械学習モデル開発の重要ポイント

## 4.5 チューニング

- 何をチューニングするか？
  - ハイパーパラメータ
- どのようにチューニングするか？
  - 交差検定法
  - グリッドサーチ

In [1]:
# 共通事前処理

# データフレーム表示用関数
from IPython.display import display
# ライブラリーのインポート
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings("ignore")

# 表示オプション
# NumPyの浮動小数点の表示精度
np.set_printoptions(suppress=True, precision=4)
# pandasの浮動小数点の表示精度
pd.options.display.float_format = "{:.4f}".format
# データフレームのすべての項目を表示
pd.set_option("display.max_columns", None)
# グラフのスタイルの設定
plt.style.use('seaborn')
# グラフのデフォルトフォント設定
plt.rcParams["font.size"] = 14
# グラフで日本語表示が可能なフォントの設定
plt.rcParams["font.family"] = 'IPAexGothic'
# 乱数の種
RANDOM_SEED = 123

サンプルデータの読み込み

In [2]:
# サンプルデータの読み込み
# (乳がん疾患データ)

# データのロード
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

# 入力データ: x (30次元)
# 正解データ: y
x = cancer.data
y = cancer.target

In [3]:
# サンプルデータの分割

# データ分割のパラメータ
from sklearn.model_selection import train_test_split

# データ分割
test_size = 0.1
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, 
    random_state=RANDOM_SEED, stratify=y)

# 分割後サイズ確認
print(x.shape)
print(x_train.shape)
print(x_test.shape)

(569, 30)
(512, 30)
(57, 30)


### 4.5.1 アルゴリズムの選択

In [4]:
# 複数アルゴリズムで精度を比較
# 結果が同じになるようrandom_stateは同一にする

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# 線形回帰
algorithm1 = LogisticRegression(random_state=RANDOM_SEED)

# サポートベクターマシン(カーネル)
algorithm2 = SVC(kernel='rbf', random_state=RANDOM_SEED)

# 決定木
algorithm3 = DecisionTreeClassifier(random_state=RANDOM_SEED)

# ランダムフォレスト
algorithm4 = RandomForestClassifier(random_state=RANDOM_SEED)

# XGBoost
algorithm5 = XGBClassifier(random_state=RANDOM_SEED)

# アルゴリズムのリスト作成
algorithms = [
    algorithm1, algorithm2, algorithm3, algorithm4, algorithm5
]

In [5]:
# 複数アルゴリズムで精度比較

for algorithm in algorithms:

    # 訓練データで学習
    algorithm.fit(x_train, y_train)

    # 検証データで精度測定
    score = algorithm.score(x_test, y_test)

    # アルゴリズム名取得
    name = algorithm.__class__.__name__

    # 精度とアルゴリズム名表示
    print(f'score: {score:.4f}  {name}')


score: 0.9649  LogisticRegression
score: 0.8947  SVC
score: 0.9474  DecisionTreeClassifier
score: 0.9298  RandomForestClassifier
score: 0.9825  XGBClassifier


### 4.5.2 ハイパーパラメータの最適化

In [6]:
# SVC デフォルトパラメータの確認
algorithm = SVC(kernel='rbf', random_state=RANDOM_SEED)
print(algorithm.get_params())

{'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': 123, 'shrinking': True, 'tol': 0.001, 'verbose': False}


In [7]:
# gammaの最適化

algorithm = SVC(kernel='rbf', random_state=RANDOM_SEED)
gammas = [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]

for gamma in gammas:
    algorithm.gamma = gamma
    algorithm.fit(x_train, y_train)
    score = algorithm.score(x_test, y_test)
    print(f'score: {score:.4f}  gamma: {gamma}')

score: 0.6316  gamma: 1
score: 0.6316  gamma: 0.1
score: 0.6316  gamma: 0.01
score: 0.9474  gamma: 0.001
score: 0.9474  gamma: 0.0001
score: 0.9474  gamma: 1e-05


In [8]:
# Cの最適化
# gammaは先ほど調べた最適値 0.001を採用

Cs = [1,  10,  100, 1000, 10000]
for C in Cs:
    algorithm = SVC(
        kernel='rbf', gamma=0.001, C=C, random_state=RANDOM_SEED
    )
    algorithm.fit(x_train, y_train)
    score = algorithm.score(x_test, y_test)
    print(f'score: {score:.4f}  C: {C}')

score: 0.9474  C: 1
score: 0.9298  C: 10
score: 0.9298  C: 100
score: 0.9298  C: 1000
score: 0.9298  C: 10000


### 4.5.3 交差検定法（Cross validation）

In [9]:
# 特定のアルゴリズムに対して交差検定を実施

# アルゴリズムの定義
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

algorithm = SVC(
    kernel='rbf', gamma=0.001, C=1, random_state=RANDOM_SEED
)

# 分割時に正解データの分布が偏らないようにStratifiedKFoldを利用
stratifiedkfold = StratifiedKFold(n_splits=3)

# 交差検定の実施 (分割数=3)
scores = cross_val_score(
    algorithm, x_train, y_train, cv=stratifiedkfold
)

# 平均値の計算
mean = scores.mean()

# 結果表示
print(f'平均スコア: {mean:.4f}  個別スコア: {scores}')

平均スコア: 0.9141  個別スコア: [0.8889 0.9181 0.9353]


In [10]:
# 候補アルゴリズムのリスト作成

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# 線形回帰
algorithm1 = LogisticRegression(random_state=RANDOM_SEED)

# サポートベクターマシン(カーネル)
algorithm2 = SVC(
    kernel='rbf', gamma=0.001, C=1, random_state=RANDOM_SEED
)

# 決定木
algorithm3 = DecisionTreeClassifier(random_state=RANDOM_SEED)

# ランダムフォレスト
algorithm4 = RandomForestClassifier(random_state=RANDOM_SEED)

# XGBoost
algorithm5 = XGBClassifier(random_state=RANDOM_SEED)

# アルゴリズムのリスト作成
algorithms = [
    algorithm1, algorithm2, algorithm3, algorithm4, algorithm5
]

In [11]:
# 複数アルゴリズムで精度を比較

# 分割時に正解データの分布が偏らないようにStratifiedKFoldを利用
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

stratifiedkfold = StratifiedKFold(n_splits=3)

for algorithm in algorithms:
    # 交差検定法の実行
    scores = cross_val_score(
        algorithm, x_train, y_train, cv=stratifiedkfold
    )
    score = scores.mean()
    name = algorithm.__class__.__name__
    print(f'平均スコア: {score:.4f}  個別スコア: {scores}  {name}')

平均スコア: 0.9492  個別スコア: [0.9474 0.9474 0.9529]  LogisticRegression
平均スコア: 0.9141  個別スコア: [0.8889 0.9181 0.9353]  SVC
平均スコア: 0.9062  個別スコア: [0.8713 0.9415 0.9059]  DecisionTreeClassifier
平均スコア: 0.9629  個別スコア: [0.9649 0.9591 0.9647]  RandomForestClassifier
平均スコア: 0.9570  個別スコア: [0.9474 0.9649 0.9588]  XGBClassifier


### 4.5.4 グリッドサーチ

In [12]:
# グリッドサーチを交差検定を組み合わせて最適なパラメータを探索
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

params = {
    'C': [1, 10, 100, 1000, 10000],
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
}
algorithm = SVC(random_state=RANDOM_SEED)

stratifiedkfold = StratifiedKFold(n_splits=3)

gs = GridSearchCV(algorithm, params, cv=stratifiedkfold)
gs.fit(x_train, y_train)

# ベストのモデルを取得し検証データを分類
best = gs.best_estimator_
best_pred = best.predict(x_test)
print(best)

SVC(C=1000, gamma=1e-05, random_state=123)


In [13]:
# 精度の取得
from sklearn.metrics import confusion_matrix

score = best.score(x_test, y_test)
print(f'スコア: {score:.4f}')

# 混同行列を出力
print()
print('混同行列')
print(confusion_matrix(y_test, best_pred))

スコア: 0.9825

混同行列
[[20  1]
 [ 0 36]]


### 4.5.5 その他のチューニング

- 特徴エンジニアリング
  - 欠損値の処理
  - 多値ラベルのOne-Hotエンコーディング
  - 正規化
  - 複数の入力項目を組み合わせて新しい項目を作る
  - etc